In [1]:
from config.config import Config
from dataloader import Dataset
import matplotlib.pyplot as plt
import numpy as np

In [2]:
cfg = Config('./config/config.yml')

/home/jovyan/nas/1_user/eunsung.shin@agilesoda.ai/module/CRAFTS/config/config.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self._dict = yaml.load(self._yaml)


In [3]:
cfg.print()

Model Configurations : 
------------------------------
EXP_NAME: ''                                                                     # Where to store logs and models
SAVED_MODEL_PATH: '/home/jovyan/nas/1_user/eunsung.shin@agilesoda.ai/module/CRAFTS/saved_models/TPS-SimpleResNet-BiLSTM-Attn-Seed123/CRAFTS15.pth' #'/home/jovyan/nas/1_user/eunsung.shin@agilesoda.ai/module/CRAFTS/saved_models/TPS-ResNet-BiLSTM-Attn-Seed123/CRAFTS28.pth'
MODE: 1                                                                          # 1: train, 2: test
MODEL: 1                                                                         # 1: CRAFTS, 2: CRAFT, 3: STR
SEED: 123                                                                       # random seed
GPU: ['2']                                                                         # list of gpu ids
WORKERS: 16                                                                      # number of data loading workers
CUDA: True # 현재, test_crafts는 무조건 cpu모드

In [8]:
ls =[]
ls.extend([1])
ls

[1]

In [ ]:
dataloader = Dataset(cfg)

In [ ]:
for i in range(6500) :
    gtfile = os.path.join(dataloader.gt_folder, dataloader.images_path[i].replace('.jpg', '.txt'))
    with open(gtfile, 'r', encoding='utf-8-sig') as f :
        data = f.read().split('\n')
        for d in data :
            if '[ROT90' in d :
                print(i)
                break
#             if len(d.split('\t')) > 9 :
#                 print(i)
#                 break

In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt

i = 3
gtfile = os.path.join(dataloader.gt_folder, dataloader.images_path[i].replace('.jpg', '.txt'))
imgfile = os.path.join(dataloader.img_folder, dataloader.images_path[i])

In [ ]:
bboxes, words = dataloader.load_gt(gtfile)

In [ ]:
from PIL import Image, ImageDraw
import numpy as np

img = Image.open(imgfile)
imgdraw = ImageDraw.Draw(img)

for bbox in bboxes :
    imgdraw.polygon(bbox.flatten().tolist(), fill = None, outline = (0,255,0))

In [ ]:
img

In [ ]:
plt.imshow(dataloader.crop_image_by_bbox(np.array(img), bboxes[5], words[1])[0])

In [ ]:
dataloader.inference_pursedo_bboxes(np.array(img), bboxes[3], words[3], viz = True)

In [ ]:
from coordinates import sort_rectangle_custom
box = sort_rectangle_custom(box)

In [ ]:
box = bboxes[3]
word = words[3]

box

In [ ]:
w = (int)(np.linalg.norm(box[0] - box[1]))
h = (int)(np.linalg.norm(box[0] - box[3]))

w,h

In [ ]:
rect = cv2.boundingRect(box)
x,y,w,h = rect

box = np.array([[x,y],
                [x+w, y],
                [x+w, y+h],
                [x, y+h]])

In [ ]:
import cv2
w = (int)(np.linalg.norm(box[0] - box[1]))
h = (int)(np.linalg.norm(box[0] - box[3]))

width = w
height = h
# if h > w * 1.5:
#     width = h
#     height = w
#     # [ROT90]일 때
#     M = cv2.getPerspectiveTransform(np.float32(box),
#                                     np.float32(np.array([[0, height], [0,0], [width, 0], [width, height]])))
# #     [ROT270]일 때
#     M = cv2.getPerspectiveTransform(np.float32(box),
#                                                 np.float32(np.array([[width, 0], [width, height], [0, height], [0, 0]])))
# else :
#     M = cv2.getPerspectiveTransform(np.float32(box),
#                                     np.float32(np.array([[0, 0], [width, 0], [width, height], [0, height]])))
rot_angle = 90
if h > w * 1.5 and len(word) != 1 :
    width = h
    height = w
    # [ROT90]일 때
#     if rot_angle == 90 or rot_angle == None :
#         M = cv2.getPerspectiveTransform(np.float32(box),
#                         np.float32(np.array([[0, height], [0,0], [width, 0], [width, height]])))
#     elif rot_angle == 270 or rot_angle == 0 :
    M = cv2.getPerspectiveTransform(np.float32(box),
                                np.float32(np.array([[width, 0], [width, height], [0, height], [0, 0]])))
#     else :
#     M = cv2.getPerspectiveTransform(np.float32(box),
#                         np.float32(np.array([[0, height], [0,0], [width, 0], [width, height]])))
# else:
#     M = cv2.getPerspectiveTransform(np.float32(box),
#                                     np.float32(np.array([[0, 0], [width, 0], [width, height], [0, height]])))

warped = cv2.warpPerspective(np.array(img), M, (width, height))

plt.imshow(warped)


In [ ]:
# polygon(>4)
image = np.array(img)
pts = np.int32(box)
rect = cv2.boundingRect(pts)
x,y,w,h = rect
x,y = max(0, x), max(0, y)
x,y = min(x, image.shape[1]), min(y, image.shape[0])
cropped = np.array(image)[y:y+h, x:x+w]
pts = pts - pts.min(axis = 0)

mask = np.zeros(cropped.shape[:2], np.uint8)
cv2.drawContours(mask, [pts], -1, (255,255,255), -1, cv2.LINE_AA)
cropped_black_bg = cv2.bitwise_and(cropped, cropped, mask=mask)
bg = np.ones_like(cropped, np.uint8)*255
cv2.bitwise_not(bg,bg, mask = mask)
cropped_white_bg = bg + cropped_black_bg

plt.imshow(cropped_white_bg)

In [ ]:
char_bboxes, confidence = dataloader.inference_pursedo_bboxes(np.array(img), bboxes[0], words[0])

In [ ]:
img = Image.open(imgfile)
imgdraw = ImageDraw.Draw(img)

for bbox in char_bboxes :
    imgdraw.polygon(bbox.flatten().tolist(), fill = None, outline = (0,255,0))

In [ ]:
img

In [ ]:
import coordinates 

char_bboxes = [coordinates.sort_rectangle_custom(bbox) for bbox in char_bboxes][::-1]

In [ ]:
warped_ls = []
for box in char_bboxes :
    plt.show()
    w = (int)(np.linalg.norm(box[0] - box[1]))
    h = (int)(np.linalg.norm(box[0] - box[3]))
    if h > w * 1.5:
        width = h
        height = w
    M = cv2.getPerspectiveTransform(np.float32(box),
                                                np.float32(np.array([[0, 0], [width, 0], [width, height], [0, height]])))
    warped = cv2.warpPerspective(np.array(img), M, (width, height))
    warped_ls.append(warped)
    plt.imshow(warped)
    print(warped.shape)

In [ ]:
plt.figure(figsize = (120, 20))
plt.imshow(np.concatenate(warped_ls, -2))

In [ ]:
with open(gtfile, 'r', encoding = 'utf-8-sig') as f :
    data = f.read().split('\n')

In [ ]:
from tqdm import tqdm
found = None
for i, file in tqdm(enumerate(dataloader.images_path)):
    with open(os.path.join(dataloader.gt_folder,file.replace('.jpg', '.txt').replace('.png', '.txt')), 'r', encoding = 'utf-8-sig') as f :
        data= f.read().split('\n')
        
        for d in data :
            if len(d.split('\t'))> 9 :
                found = True
        if found :
            print(i)
        found = False
        

In [ ]:
image, region_scores_torch, affinity_scores_torch, confidence_mask_torch, orientation_x_torch, orientation_y_torch, cropped_word_bboxes, encoded_cropped_words, words_length = dataloader.pull_item(1)

In [ ]:
import data_utils
from dataset_CRAFTS import RegionAlignCollate
from utils import AttnLabelConverter

def load_character_list(path) :
    with open(path, 'r') as f :
        character_list = f.read()
        
    return character_list

img = image.permute(1,2,0).detach().numpy()
character_list = load_character_list('./char_dicts/charset-kor-sp.txt')
converter = AttnLabelConverter(character_list)

words = encoded_cropped_words
word_bboxes = cropped_word_bboxes
words_length = words_length
decoded_words = converter.decode(words, words_length)


feature_batch, text_batch, length_batch = [],[],[]

padding = RegionAlignCollate(imgH=16, imgW=64, keep_ratio_with_pad = False)
for word_bbox, word, decoded_word, word_length in zip(word_bboxes, words, decoded_words, words_length) :

    if word_length != 1  :
        cropFeature, _ = data_utils.crop_image_by_bbox(img, word_bbox*2, decoded_word)
#                         xmin, xmax = int(torch.min(word_bbox[:,0])), int(torch.max(word_bbox[:,0]))
#                         ymin, ymax = int(torch.min(word_bbox[:,1])), int(torch.max(word_bbox[:,1]))
#                         if xmax - xmin == 0 or ymax - ymin == 0 :
#                             print('found zero-size image...skipping.')
#                             print(word_bbox)
#                             continue
#                         cropped = STR_input[:,ymin:ymax, xmin:xmax]




#                         try : 
        feature_batch.append(cropFeature)
#         text_batch.append(word.unsqueeze(0))
#         length_batch.append(word_length.unsqueeze(0))
#                         except :
#                             print(word_bbox)
#                             print('idx : ', index)

pad_batch = padding(feature_batch)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(pad_batch[3].permute(1,2,0).detach()*255.)

In [ ]:
from imgproc import cvt2HeatmapImg
plt.figure(figsize = (15, 15))
plt.imshow(cvt2HeatmapImg(np.array(region_scores_torch)))

In [ ]:
from imgproc import cvt2HeatmapImg
plt.figure(figsize = (15, 15))
plt.imshow(cvt2HeatmapImg(np.array(affinity_scores_torch)))

In [ ]:
plt.figure(figsize = (15, 15))
plt.imshow(cvt2HeatmapImg(np.array(orientation_x_torch)))

In [ ]:
plt.figure(figsize = (15, 15))
plt.imshow(cvt2HeatmapImg(np.array(orientation_y_torch)))

# Visualize Orientation

In [ ]:


def scale_orientation(orientation_x, orientation_y, region_scores) :
    orientation_region = np.arctan(np.array(orientation_y) / np.array(orientation_x) )*255.
    mask = np.zeros_like(orientation_x)
    
    mask[np.isnan(orientation_region)] = 0
    mask[~np.isnan(orientation_region)] = 255
    return np.uint8(orientation_region) , np.uint8(mask)

In [ ]:
import cv2

orientation_x = np.array(orientation_x_torch)*255.
orientation_y = np.array(orientation_y_torch)*255.
region_score = np.array(region_scores_torch)*255.

orientation_h, mask = scale_orientation(orientation_x, orientation_y, region_score)


# orientation_s
orientation_s = region_score.copy()
orientation_s[orientation_s <255.*0.2] = 0
orientation_s = np.uint8(orientation_s)

# orientation_v
scale_region = orientation_s.copy()
scale_region[scale_region<255.*0.3] = 0
orientation_v = scale_region


hsv = cv2.merge([orientation_h, orientation_s, orientation_v])

hsv = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


plt.figure(figsize = (15,15))
plt.imshow(hsv)

In [ ]:
import cv2, torch
img = torch.randn((1920,1440, 130))
w = (int)(np.linalg.norm(box[0] - box[1]))
h = (int)(np.linalg.norm(box[0] - box[3]))
width = w
height = h
if h > w * 1.5:
    width = h
    height = w
    # [ROT90]일 때
    M = cv2.getPerspectiveTransform(np.float32(box),
                                    np.float32(np.array([[0, height], [0,0], [width, 0], [width, height]])))
#     [ROT270]일 때
    M = cv2.getPerspectiveTransform(np.float32(box),
                                                np.float32(np.array([[width, 0], [width, height], [0, height], [0, 0]])))
else :
    M = cv2.getPerspectiveTransform(np.float32(box),
                                    np.float32(np.array([[0, 0], [width, 0], [width, height], [0, height]])))

warped = cv2.warpPerspective(np.array(img), M, (width, height))

# plt.imshow(warped)


In [ ]:
plt.imshow(warped[:,:,:3])

In [ ]:
encoded_cropped_words[0]

In [ ]:
import torch

data_loader = torch.utils.data.DataLoader(dataloader,
                                              batch_size=cfg.BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=cfg.WORKERS,
                                              drop_last=True,
                                              pin_memory=True)

In [ ]:
from tqdm import tqdm
for index, (images, gh_label, gah_label, mask, ori_x, ori_y, word_bboxes_batch, words_batch, words_length_batch) in tqdm(enumerate(data_loader)):
    print(words_batch)
    break

In [ ]:
words_length_batch

In [ ]:
words

In [ ]:
import re
class AttnLabelConverter(object):
    """ Convert between text-label and text-index """

    def __init__(self, character):
        # character (str): set of the possible characters.
        # [GO] for the start token of the attention decoder. [s] for end-of-sentence token.
        list_token = ['[GO]', '[s]', '[UNK]']  # ['[s]','[UNK]','[PAD]','[GO]']
        list_character = list(character)
        self.character = list_token + list_character
        self.stop_token = list_token[-2]

        self.dict = {}
        for i, char in enumerate(self.character):
            # print(i, char)
            self.dict[char] = i

    def encode(self, text, batch_max_length=25):
        """ convert text-label into text-index.
        input:
            text: text labels of each image. [batch_size]
            batch_max_length: max length of text label in the batch. 25 by default

        output:
            text : the input of attention decoder. [batch_size x (max_length+2)] +1 for [GO] token and +1 for [s] token.
                text[:, 0] is [GO] token and text is padded with [GO] token after [s] token.
            length : the length of output of attention decoder, which count [s] token also. [3, 7, ....] [batch_size]
        """
        text = [re.sub(r'[UNK[0-9]+]|[ROT[0-9]+]', '', t).strip() for t in text]
#         text = [t.strip() for t in text]
        length = [len(s) + 1 for s in text]  # +1 for [s] at end of sentence.
        # batch_max_length = max(length) # this is not allowed for multi-gpu setting
        batch_max_length += 1
        # additional +1 for [GO] at first step. batch_text is padded with [GO] token after [s] token.
        batch_text = torch.zeros((len(text), batch_max_length + 1), dtype=torch.int64)
        for i, t in enumerate(text):
            text = list(t)
            text.append(self.stop_token)
            text = [self.dict[char] if char in self.dict\
                                    else self.dict['[UNK]']\
                                    for char in text ]
            batch_text[i][1:1 + len(text)] = torch.LongTensor(text)  # batch_text[:, 0] = [GO] token

        return (batch_text, np.array(length))

    def decode(self, text_index, length):
        """ convert text-index into text-label. """
        texts = []
        for index, l in enumerate(length):
            text = ''.join([self.character[i] for i in text_index[index, :]])
            texts.append(text)
        return texts

def load_character_list(path) :
    with open(path, 'r') as f :
        character_list = f.read()
        
    return character_list
character_list = load_character_list('./char_dicts/charset-kor-sp.txt')
converter = AttnLabelConverter(character_list)


In [ ]:
len(''.join(converter.character)) - 12

In [ ]:


plt.imshow(images[0].permute(1,2,0))

In [ ]:
text = re.sub(r'[UNK[0-9]+]|[ROT[0-9]+]', '', text)
text = [t.strip() for t in text]
length = [len(s) + 1 for s in text] 

length

In [ ]:
samp_batch_text, samp_length = converter.encode(words)

In [ ]:
og_image, character_bboxes, words, confidence_mask, confidences, word_bboxes, og_shape = dataloader.load_image_gt_and_confidencemask(0)
words

In [ ]:
words_batch[0].shape

In [ ]:
idx = 0
word_bboxes = word_bboxes_batch[i]
words = words_batch[idx]
words_length = words_length_batch[idx]

for word_bbox, word, word_length in zip(word_bboxes, words, words_length) :
                    
    if word_length != 1  :
        data_utils.crop_image_by_bbox()

In [ ]:
converter.decode(words_batch[0], words_length_batch[0])

In [ ]:
import numpy as np
converter.decode(words_batch[0], words_length_batch[0])

In [ ]:
import re

samp= '[ROT0]sdfsd'

samp = re.sub(r'[UNK[0-9]+]|[ROT[0-9]+]', '', samp)

In [ ]:
np.random.choice()

In [ ]:
def isInterleave(s1: str, s2: str, s3: str) -> bool:
    s1 = list(s1)
    s2 = list(s2)

    for char in s3 :
        if char == s1[0] :
            s1.pop(0)
        elif char == s2[0] :
            s2.pop(0)
        else :
            return False
        print(char)

In [ ]:
def isInterleave(s1: str, s2: str, s3: str) -> bool:
    i,j = 0,0
    for char in s3 :
        if s1[i] == char :
            i+=1
        elif s2[j] == char :
            j+=1
        else :
            return False
        if i == len(s1) or j == len(s2) :
            return False
            
        print(char)

In [ ]:
def isInterleave(s1: str, s2: str, s3: str) -> bool:
    s3 = list(s3)
    for i in range(len(s1)) :
        char = s3.pop(0)
        if s1[i] == char or s2[i] == char :
            continue
        else :
            return False
            

In [ ]:
s1 = 'aabcc'
s2 = 'dbbca'
s3 = 'aadbbcbcac'
isInterleave(s1,s2,s3)